# Stylised Facts

Statistical analysis of simulated exchange events: event distributions,
inter-arrival times, return distributions, and volatility clustering.

## Data generation

```bash
./build/qrsdp_run --seed 42 --days 5 --seconds 23400
```

In [1]:
from pathlib import Path

import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from scipy import stats

import qrsdp_reader as reader
import book_replay as replay
import ohlc

In [2]:
# --- Configuration ---
RUN_DIR = Path("../output/run_42")
DAY_INDEX = 0  # which day to analyse (0 = first)

manifest = reader.load_manifest(RUN_DIR)
session = manifest["sessions"][DAY_INDEX]
day_file = RUN_DIR / session["file"]
header = reader.read_header(day_file)
events = reader.read_day(day_file)

print(f"Date: {session['date']}")
print(f"Events: {len(events):,}")
print(f"Session: {header['session_seconds']}s, p0 = {header['p0_ticks']}")

Date: 2026-01-02
Events: 2,262,506
Session: 23400s, p0 = 10000


## Event Type Distribution

In [3]:
type_names = [reader.EVENT_TYPES.get(t, f"UNKNOWN_{t}") for t in range(6)]
type_counts = np.bincount(events["type"], minlength=6)

fig_types = go.Figure(
    data=go.Bar(
        x=type_names,
        y=type_counts[:6],
        marker_color=["#2196F3", "#FF5722", "#90CAF9", "#FFAB91", "#4CAF50", "#F44336"],
    ),
    layout=go.Layout(
        title="Event Type Distribution",
        xaxis=dict(title="Event Type"),
        yaxis=dict(title="Count"),
        height=400,
        template="plotly_white",
    ),
)
fig_types.show()

for name, count in zip(type_names, type_counts[:6]):
    print(f"  {name:>14s}: {count:>10,}  ({100 * count / len(events):.1f}%)")

         ADD_BID:    515,055  (22.8%)
         ADD_ASK:    514,506  (22.7%)
      CANCEL_BID:    170,963  (7.6%)
      CANCEL_ASK:    170,892  (7.6%)
     EXECUTE_BUY:    444,760  (19.7%)
    EXECUTE_SELL:    446,330  (19.7%)


## Inter-Arrival Time Distribution

For a Poisson process, inter-arrival times should follow an exponential distribution.

In [4]:
ts = events["ts_ns"].astype(np.float64)
iat_ns = np.diff(ts)
iat_us = iat_ns / 1e3  # microseconds

# Fit exponential
mean_iat = np.mean(iat_us)
rate = 1.0 / mean_iat

# Histogram with exponential overlay
hist_vals, bin_edges = np.histogram(iat_us, bins=200, range=(0, np.percentile(iat_us, 99)))
bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
bin_width = bin_edges[1] - bin_edges[0]

exp_fit = len(iat_us) * bin_width * rate * np.exp(-rate * bin_centers)

fig_iat = go.Figure()
fig_iat.add_trace(go.Bar(
    x=bin_centers, y=hist_vals, name="Observed",
    marker_color="#2196F3", opacity=0.7,
))
fig_iat.add_trace(go.Scatter(
    x=bin_centers, y=exp_fit, name=f"Exponential fit (λ={rate:.4f}/μs)",
    mode="lines", line=dict(color="red", width=2),
))
fig_iat.update_layout(
    title="Inter-Arrival Time Distribution",
    xaxis=dict(title="Inter-arrival time (μs)"),
    yaxis=dict(title="Count"),
    height=400,
    template="plotly_white",
)
fig_iat.show()

print(f"Mean IAT: {mean_iat:.2f} μs")
print(f"Median IAT: {np.median(iat_us):.2f} μs")
print(f"Event rate: {rate:.4f} events/μs = {rate * 1e6:.0f} events/s")

Mean IAT: 10342.52 μs
Median IAT: 7147.26 μs
Event rate: 0.0001 events/μs = 97 events/s


## Return Distributions at Multiple Scales

Mid-price returns at 1s, 10s, and 60s horizons. Real markets show fat tails
relative to a normal distribution.

In [5]:
book_data = replay.replay_book(
    events,
    p0_ticks=header["p0_ticks"],
    levels_per_side=header["levels_per_side"],
    initial_spread_ticks=header["initial_spread_ticks"],
    initial_depth=header["initial_depth"],
)

bars_all = ohlc.multi_resolution_ohlc(book_data["ts_ns"], book_data["mid_ticks"])

In [6]:
fig_ret = go.Figure()
colors = {"1s": "#2196F3", "10s": "#FF9800", "1min": "#4CAF50"}

for label in ["1s", "10s", "1min"]:
    df = bars_all[label]
    returns = df["close"].diff().dropna().values
    if len(returns) == 0:
        continue
    
    # Normalise for comparison
    mu, sigma = returns.mean(), returns.std()
    if sigma > 0:
        z = (returns - mu) / sigma
    else:
        z = returns
    
    fig_ret.add_trace(go.Histogram(
        x=z, name=f"{label} returns",
        marker_color=colors[label], opacity=0.5,
        nbinsx=100, histnorm="probability density",
    ))
    
    kurt = stats.kurtosis(z)
    print(f"  {label:>4s}: n={len(returns):>6,}, σ={sigma:.4f}, kurtosis={kurt:.2f}")

# Normal reference
x_norm = np.linspace(-5, 5, 200)
fig_ret.add_trace(go.Scatter(
    x=x_norm, y=stats.norm.pdf(x_norm),
    name="N(0,1)", mode="lines",
    line=dict(color="black", width=2, dash="dash"),
))

fig_ret.update_layout(
    title="Standardised Return Distributions (check for fat tails)",
    xaxis=dict(title="Standardised return"),
    yaxis=dict(title="Density"),
    barmode="overlay",
    height=450,
    template="plotly_white",
)
fig_ret.show()

    1s: n=23,399, σ=0.8580, kurtosis=0.78
   10s: n= 2,339, σ=2.6699, kurtosis=0.23
  1min: n=   389, σ=6.2279, kurtosis=0.02


## Return Autocorrelation

- **Returns** autocorrelation should be near zero (weak-form efficiency).
- **Absolute returns** autocorrelation should be positive (volatility clustering).

In [7]:
def autocorrelation(x, max_lag=50):
    """Compute autocorrelation for lags 1..max_lag."""
    x = x - x.mean()
    var = np.var(x)
    if var == 0:
        return np.zeros(max_lag)
    result = np.correlate(x, x, mode="full")
    result = result[len(x) - 1:]  # positive lags only
    result = result / (var * len(x))
    return result[1:max_lag + 1]


returns_10s = bars_all["10s"]["close"].diff().dropna().values
abs_returns_10s = np.abs(returns_10s)

max_lag = 50
acf_ret = autocorrelation(returns_10s, max_lag)
acf_abs = autocorrelation(abs_returns_10s, max_lag)
lags = np.arange(1, max_lag + 1)

fig_acf = go.Figure()
fig_acf.add_trace(go.Bar(
    x=lags, y=acf_ret, name="Returns ACF",
    marker_color="#2196F3", opacity=0.7,
))
fig_acf.add_trace(go.Bar(
    x=lags, y=acf_abs, name="|Returns| ACF",
    marker_color="#FF5722", opacity=0.7,
))

# 95% confidence band
ci = 1.96 / np.sqrt(len(returns_10s))
fig_acf.add_hline(y=ci, line_dash="dash", line_color="grey", annotation_text="95% CI")
fig_acf.add_hline(y=-ci, line_dash="dash", line_color="grey")

fig_acf.update_layout(
    title="Autocorrelation: Returns vs |Returns| (10s bars)",
    xaxis=dict(title="Lag"),
    yaxis=dict(title="ACF"),
    barmode="group",
    height=400,
    template="plotly_white",
)
fig_acf.show()

## Price Shift Frequency Over Time

Rolling count of events where the best price changed, measured in 1-minute windows.

In [8]:
# Identify price shifts: events where best_bid or best_ask changed
bid_changes = np.diff(book_data["best_bid"]) != 0
ask_changes = np.diff(book_data["best_ask"]) != 0
shifts = bid_changes | ask_changes

# Bin into 1-minute windows
ts_s = book_data["ts_ns"][1:] / 1e9  # skip first to match diff length
window_ns = 60_000_000_000  # 1 minute
t0 = int(book_data["ts_ns"][0])
bins = ((book_data["ts_ns"][1:].astype(np.int64) - t0) // window_ns).astype(np.int64)

shift_df = pd.DataFrame({"bin": bins, "shift": shifts.astype(int)})
shift_counts = shift_df.groupby("bin")["shift"].sum()
shift_times = t0 / 1e9 + shift_counts.index.values * 60

fig_shifts = go.Figure(
    data=go.Scatter(
        x=shift_times, y=shift_counts.values,
        mode="lines", name="Shifts/min",
        line=dict(color="#9C27B0"),
    ),
    layout=go.Layout(
        title="Price Shift Frequency (1-min rolling)",
        xaxis=dict(title="Time (s)", rangeslider=dict(visible=True)),
        yaxis=dict(title="Shifts per minute"),
        height=350,
        template="plotly_white",
    ),
)
fig_shifts.show()

total_shifts = shifts.sum()
print(f"Total price shifts: {total_shifts:,} out of {len(events):,} events ({100 * total_shifts / len(events):.2f}%)")

Total price shifts: 40,680 out of 2,262,506 events (1.80%)
